
# Netflix Dataset Analysis — FIXED Template (Matches Your Columns)

**Columns detected:** `Show_Id, Category, Title, Director, Cast, Country, Release_Date, Rating, Duration, Type, Description`

This notebook is aligned to these exact names and will run without renaming.


In [ ]:

# Imports & settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', None)

DATA_PATH = "/mnt/data/Netflix Dataset.csv"
df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
df.head(5)


In [ ]:

# Standardize/clean columns using the ACTUAL names:
df_clean = df.copy()

# Strip whitespace on common text cols
for col in ['Title','Director','Cast','Country','Rating','Type','Description','Category','Release_Date']:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).str.strip()

# Parse Release_Date -> datetime and year_added/month_added
df_clean['date_added'] = pd.to_datetime(df_clean['Release_Date'], errors='coerce')
df_clean['year_added'] = df_clean['date_added'].dt.year
df_clean['month_added'] = df_clean['date_added'].dt.month

# Missing values handling
for col in ['Director','Cast','Country']:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].replace({'nan': np.nan}).fillna('Unknown')

# Remove duplicates
df_clean = df_clean.drop_duplicates()

print("Cleaned shape:", df_clean.shape)
df_clean.head(3)



## Movies vs TV Shows (Category)
- Uses **Category** (`Movie` vs `TV Show`)
- Over-time trends use **year_added** because production year is not present; we use the date titles were added to Netflix.


In [ ]:

# Overall split
if 'Category' in df_clean.columns:
    split_counts = df_clean['Category'].value_counts()
    display(split_counts)
    split_counts.plot(kind='bar', title='Overall: Movies vs TV Shows')
    plt.xlabel('Category')
    plt.ylabel('Count')
    plt.show()
else:
    print("Column 'Category' not found.")


In [ ]:

# Trend over time by year_added
if set(['Category','year_added']).issubset(df_clean.columns):
    yearly = (df_clean.dropna(subset=['year_added'])
                        .groupby(['year_added','Category'])
                        .size()
                        .unstack(fill_value=0)
                        .sort_index())
    display(yearly.tail(10))
    yearly.plot(kind='line', title='Movies vs TV Shows by Year Added')
    plt.xlabel('Year Added')
    plt.ylabel('Count')
    plt.show()
else:
    print("Columns missing for yearly trend.")



## Genres (`Type` column)
- The `Type` column contains comma-separated genres (e.g., "Dramas, International Movies").


In [ ]:

def split_explode_count(series, sep=','):
    s = series.dropna().astype(str).str.split(sep).explode().str.strip()
    s = s[s!='']
    return s.value_counts()

if 'Type' in df_clean.columns:
    genre_counts = split_explode_count(df_clean['Type'])
    display(genre_counts.head(20))
    genre_counts.head(15).sort_values(ascending=True).plot(kind='barh', title='Top Genres (Overall)')
    plt.xlabel('Count'); plt.ylabel('Genre'); plt.show()
else:
    print("Column 'Type' not found.")


In [ ]:

# Genre trends by year_added for top genres
TOP_N = 8
if set(['Type','year_added']).issubset(df_clean.columns):
    exploded = df_clean.dropna(subset=['year_added']).copy()
    exploded['genre'] = exploded['Type'].str.split(',')
    exploded = exploded.explode('genre')
    exploded['genre'] = exploded['genre'].str.strip()
    exploded = exploded[exploded['genre']!='']
    top_genres = exploded['genre'].value_counts().head(TOP_N).index.tolist()
    subset = exploded[exploded['genre'].isin(top_genres)]
    genre_year = subset.groupby(['year_added','genre']).size().unstack(fill_value=0).sort_index()
    display(genre_year.tail(10))
    for g in top_genres:
        genre_year[g].plot(kind='line', title=f"Trend: {g} by Year Added")
        plt.xlabel('Year Added'); plt.ylabel('Count'); plt.show()
else:
    print("Columns missing for genre trend.")



## Country-wise Contributions (`Country`)


In [ ]:

if 'Country' in df_clean.columns:
    country_counts = split_explode_count(df_clean['Country'])
    display(country_counts.head(20))
    country_counts.head(15).sort_values(ascending=True).plot(kind='barh', title='Top Countries (Overall)')
    plt.xlabel('Count'); plt.ylabel('Country'); plt.show()
else:
    print("Column 'Country' not found.")


In [ ]:

# Country trends by year_added
TOP_N = 8
if set(['Country','year_added']).issubset(df_clean.columns):
    exploded = df_clean.dropna(subset=['year_added']).copy()
    exploded['country_item'] = exploded['Country'].str.split(',')
    exploded = exploded.explode('country_item')
    exploded['country_item'] = exploded['country_item'].str.strip()
    exploded = exploded[exploded['country_item']!='']
    top_countries = exploded['country_item'].value_counts().head(TOP_N).index.tolist()
    subset = exploded[exploded['country_item'].isin(top_countries)]
    country_year = subset.groupby(['year_added','country_item']).size().unstack(fill_value=0).sort_index()
    display(country_year.tail(10))
    for c in top_countries:
        country_year[c].plot(kind='line', title=f"Trend: {c} by Year Added")
        plt.xlabel('Year Added'); plt.ylabel('Count'); plt.show()
else:
    print("Columns missing for country trends.")



## Ratings & Duration
- **Rating** distribution
- **Duration** parsing (minutes vs seasons)


In [ ]:

# Rating distribution
if 'Rating' in df_clean.columns:
    rc = df_clean['Rating'].value_counts(dropna=False)
    display(rc)
    rc.plot(kind='bar', title='Rating Distribution')
    plt.xlabel('Rating'); plt.ylabel('Count'); plt.show()
else:
    print("Column 'Rating' not found.")


In [ ]:

# Duration parsing
if 'Duration' in df_clean.columns and 'Category' in df_clean.columns:
    durations = df_clean['Duration'].astype(str).str.extract(r'(?P<num>\d+)\s*(?P<unit>\w+)', expand=True)
    durations['num'] = pd.to_numeric(durations['num'], errors='coerce')
    df_clean['duration_num'] = durations['num']
    df_clean['duration_unit'] = durations['unit']

    movies = df_clean[(df_clean['Category']=='Movie') & (df_clean['duration_unit'].str.lower().str.startswith('min'))]
    if not movies.empty:
        movies['duration_num'].plot(kind='hist', bins=20, title='Movies: Duration (Minutes)')
        plt.xlabel('Minutes'); plt.ylabel('Frequency'); plt.show()

    shows = df_clean[(df_clean['Category']=='TV Show') & (df_clean['duration_unit'].str.lower().str.startswith('season'))]
    if not shows.empty:
        shows['duration_num'].plot(kind='hist', bins=10, title='TV Shows: Number of Seasons')
        plt.xlabel('Seasons'); plt.ylabel('Frequency'); plt.show()
else:
    print("Columns missing for duration analysis.")



## Key Findings (fill in after running)
- **Movies vs TV Shows:** …  
- **Top Genres:** …  
- **Top Countries:** …  
- **Ratings & Duration:** …  
